**NOTE:** This file contains a few command line code pieces documented for the sake of code reproducibility. Most of the scripts, except for those used for read correction, were launched from the command line directly; respective commands are represented in the *bash*-flavored chunks. Also, several commands and as well as manual file editing lines are not documented here and are described in the manuscript and/or in the RMarkdown notebook.

<div align="center"><h2>08.11.18</h2></div>
<b>22:37</b>: running <i>rcorrector.sh</i>:

In [10]:
import os
import subprocess
import codecs

In [4]:
os.chdir('/home/reverend_casy/tmp_fs')
with open('rcorrector_script.sh', 'r') as handle:
    for line in handle.readlines():
        print(line)

#! /bin/bash

mkdir rcorrected

for dir in $(ls trimmed_loose); do if [[ $dir == 1 || $dir == 5 ]]; then

mkdir rcorrected/$dir

R1=$(ls trimmed_loose/$dir | grep 'R1')

R2=$(ls trimmed_loose/$dir | grep 'R2') 

perl ~/rcorrector/run_rcorrector.pl -1 trimmed_loose/$dir/$R1 -2 trimmed_loose/$dir/$R2 -od rcorrected/$dir -t 72

fi; done



In [ ]:
#subprocess.run('./rcorrector_script.sh', shell=True)

<b>00:01:</b>: merging uncorrected reads:
```
for dir in $(ls trimmed_loose); do if [[ $dir == 1 || $dir == 5 ]]; then find ./trimmed_loose/$dir -name '*R1*' -exec cat {} >> merged_uncorrected/R1_uncorr.fastq; fi; done
```
The same was then performed with R2 reads.

<b>01:40:</b> correcting merged reads.

In [11]:
with codecs.open('rcorrector_script_2.sh', 'r', encoding='utf-8',
                 errors='ignore') as handle:
    for line in handle.readlines():
        print(line)

#! /bin/bash

mkdir merged_rcorrected

R1=$(ls merged_uncorrected | grep 'R1')

R2=$(ls merged_uncorrected | grep 'R2') 

echo merged_uncorrected/$R1

perl ~/rcorrector/run_rcorrector.pl -1 merged_uncorrected/$1 -2 merged_uncorrected/$R2 -od merged_rcorrected -t 72



<h3>Jellyfish k-mer estimation</h3>
```bash

mkdir jellyfish_stats

cat ../merged_uncorrected/R1_uncorr.fastq ../merged_uncorrected/R2_uncorr.fastq > uncorr.concat.fasta

cat ../merged_rcorrected/R1_uncorr.cor.fq ../merged_rcorrected/R2_uncorr.cor.fq > corr.concat.fasta


```

<div align='center'><h2>09.11.19</h2></div>

<b>16:52:</b> Running Trinity assembly on the corrected merged reads:
```
Trinity --seqrType fq --righ merged_rcorrected/R1_uncorr.cor.fq --right merged_rcorrected/R2_uncorr.cor.fq -max_memory 100G -CPU 72 --output trinity_merged_rcorrected_assembly
```

In [24]:
from collections import defaultdict
# from operator import attrgetter
from Bio import SeqIO

def read_fasta(file:str)->list:
    with open(file, 'r') as handle:
        seq_list = []
        for entry in SeqIO.parse(handle, 'fasta'):
            seq_list.append(entry)
    return seq_list

def filter_empty(contig_list:list, protein_list:list)->list:
    """
    wipe out contigs without any coding sequence embedded or coding sequence lost after CD-Hit clustering
    """
    contig_list = [contig for contig in contig_list if contig.id in map(lambda x: x.id[:-3], protein_list)]
    return contig_list
    # works properly


def filter_assembly(contig_list:list, protein_list:list)->list:
    """
    remove contigs which fail to contain a plausible coding sequence
    """
    
    def filter_short_contigs(contig_list:list)->list:
        contig_list = [contig for contig in contig_list if len(contig.seq) >= 300]
        return contig_list
    # works properly
    
    def create_dict(contig_list:list, protein_list:list)->dict:
        """
        devise a list dictionary to store all proteins 
        note that since SeqRecord objects are unhashable we use contig IDs as dictionary keys
        """
        contig_dict = defaultdict(list)
        for contig in contig_list:
            for protein in protein_list:
                if protein.id[:-3] == contig.id
                    contig_dict[contig.id].append(protein)
                    
        return contig_dict
    # works properly
    
    
    def leave_longest(contig_dict:dict)->dict:
        """
        leave only the longest CDS if more than one ORFs are found
        """
        contig_dict = dict(zip(contig_dict.keys(),
                           map(lambda x: max(x, key=lambda y:len(y.seq)), contig_dict.values())))
        return contig_dict
    # works properly
    
    def filter_short_orfs(contig_dict:dict)->dict:
        """
        remove dictionary entries whose ORFs are shorter than 300bp
        """
        contig_dict = {k: v for k, v in contig_dict.items() if len(v.seq) >= 100}
        return contig_dict
    # works properly
        
    def filter_overhangs(contig_dict:dict)->dict:
        """
        remove contigs whose coding sequences cover less than 30% of their total length
        """
        contig_dict = {k:v for k,v in contig_dict.items() if len(v.seq) >=
                       len(next((x for x in contig_list if x.id == k), None).seq)*0.3}
        return contig_dict
    
    
     def create_dict(contig_list:list, protein_list:list)->dict:
        """
        devise a list dictionary to store all proteins 
        note that since SeqRecord objects are unhashable we use contig IDs as dictionary keys
        """
        contig_dict = defaultdict(list)
        for contig in contig_list:
            for protein in protein_list:
                if protein.id[:-3] == contig.id
                    if protein
                    contig_dict[contig.id].append(protein)
                    
        return contig_dict
    # works properly
    
    
    contig_list = filter_empty(contig_list, protein_list)
    contig_list = filter_short_contigs(contig_list)
    contig_dict = create_dict(contig_list, protein_list)
    contig_dict = leave_longest(contig_dict)
    contig_dict = filter_short_orfs(contig_dict)
    contig_dict = filter_overhangs(contig_dict)
    
    contigs = (contig for contig in contig_list if contig.id in contig_dict.keys())
    
    return contigs, contig_dict.values()

def write(seqs:list, file:str):
    with open(file, 'w') as handle:
        SeqIO.write(seqs, handle, 'fasta')

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 72)

In [ ]:

def read_fasta(file:str)->list:
    with open(file, 'r') as handle:
        seq_list = []
        for entry in SeqIO.parse(handle, 'fasta'):
            seq_list.append(entry)
    return seq_list


def filter_empty(contig_list:list, protein_list:list)->list:
    """
    wipe out contigs without any coding sequence embedded or coding sequence lost after CD-Hit clustering
    """
    contig_list = [contig for contig in contig_list if contig.id in map(lambda x: x.id[:-3], protein_list)]
    return contig_list


def filter_assembly(contig_list:list, protein_list:list)->list:
    """
    remove contigs which fail to contain a plausible coding sequence
    """
    
     def major_filter(contig_list:list, protein_list:list)->dict:
        """
        a major filtering function replacing filter_short_contig, filter_short_orfs and filte_overhangs functions
        """
        contig_dict = defaultdict(list)
        for contig in contig_list:
            if len(contig.seq) > 300: ## replaces filter_short_contigs in the first place
                for protein in protein_list:
                    if protein.id[:-3] == contig.id
                        if len(protein.seq)*3 >= len(contig.seq)*0.3 and len(protein.seq) >= 100 
                        ## replaces filter_short_orfs and filter_overhangs
                        contig_dict[contig.id].append(protein)
                    
        return contig_dict
        
    def leave_longest(contig_dict:dict)->dict:
        """
        leave only the longest CDS if more than one ORFs are found
        """
        contig_dict = dict(zip(contig_dict.keys(),
                           map(lambda x: max(x, key=lambda y:len(y.seq)), contig_dict.values())))
        return contig_dict    
    
    contig_list = filter_empty(contig_list, protein_list)
    contig_dict = major_filter(contig_list, protein_list)
    contig_dict = leave_longest(contig_dict)    
    contigs = (contig for contig in contig_list if contig.id in contig_dict.keys())
    
    return contigs, contig_dict.values()

<b>05:54:</b> running TransDecoder on the obtained assembly:
```bash
cd transdecoder_merged_rcorrected
TransDecoder.LongOrfs -t trinity_merged_rcorrected_assembly/Trinity.fasta
TransDecoder/Predict -t trinity_merged_rcorrected_assembly/Trinity.fasta
```

<b>23:23:</b>filtering assembly with the <i>filter_assembly</i> function

In [ ]:
#contig_list, protein_list = read_fasta('trinity_merged_rcorrected_assembly/Trinity.fasta'), read_fasta('transdecoder_merged_rcorrected/Trinity.fasta.transdecoder.pep')

<div align="center"><h2>11.11.18</h2></div>
<b>19:56</b>: running <i>filter_script.py</i> from the command line because of the local ssshfs mount breakdown:

```bash
chmod +x filter_script.py
mkdir filtered
python filter_script.py -nu trinity_merged_rcorrected_assembly/Trinity.fasta -pr transdecoder_merged_rcorrected/Trinity.fasta.transdecoder.pep -o filtered
```

<b>06:14:</b> a resulting set of 9605 contigs and respective proteins has been obtained

In [16]:
def cdhit_correction(contigs:str, proteins:str, cd_output:str, contig_output:str, threshold:int)->list:
    ## first cluster proteins in proteins.fa with CD-Hit with a provided identity threshold
    ## then use the resulting file to /filter_empty/ onto the contigs.fa
    ## finally, write the filtered contig list to the file with /write/ function
    
    DEVNULL = open(os.devnull, 'w')
    
    def is_tool(name)->bool:
        """
        Check whether `name` is on PATH
        """

        from distutils.spawn import find_executable

        return find_executable(name) is not None
    
    def cd_hit(proteins:str, cd_output:str, threshold:int):
        """
        run CD-Hit for protein sequences if the tool is in $PATH and write the output to the specified file
        """
    
        if is_tool('cd-hit'):
            subprocess.run(f'cd-hit -i {proteins} -o {cd_output} -c {theshold} -n 5 -d 0 -M 0 -T 72', shell=True,
                          stdout = DEVNULL, stderr = DEVNULL)
            
    def filter_missing(contigs:str, filtered_proteins:str)->list:
        """
        wipe out contigs whose respective proteins were filtered on CD-Hit clustering stage
        """
        
        contig_list, protein_list = read_fasta(contigs), read_fasta(filtered_proteins)
        contig_list = [contig for contig in contig_list if contig.id in map(lambda x: x.id[:-3], protein_list)]
        
        return contig_list
    
    cd_hit(proteins, cd_output, threshold)
    write(filter_missing(contigs, cd_output), contig_output)

<b>06:30:</b> downloading IPG entries for hugher plants
```bash

    esearch -db ipg -query "(Viridiplantae[Organism] OR viridiplantae[All Fields]) AND plants_and_fungi[filter]" | efetch -format fasta > blastdb/plant_seqs.fasta

```

Several functions coming in handy during <i>filter_script.py</i> debugging:

In [2]:
def find_by_length(seqs:list, cutoff:int): ### implemented in find_by_length.py
    return (seq for seq in seqs if len(seq.seq) >= cutoff)

def mean_length(seqs:list)->str:
    shortest = len(min(seqs, key = lambda x: len(x.seq)).seq)
    longest = len(max(seqs, key = lambda x: len(x.seq)).seq)
    mean = sum(list(map(lambda x: len(x.seq), seqs))) / len(seqs)
    
    return f'The longest sequence is {longest} symbols long; the shortest sequence is {shortest} symbols long; mean sequence length across the file is {mean}'



<p>Launching of CD-Hit correction with <b>0.9</b> identity cutoff after application of the filtering script has resulted in <b>25756</b> contig-protein pairs</p><p>Estimating N50 statistics for every recorded assembly step:</p>

```bash

TrinityStats.pl Trinity.fasta

```

<h2>Quality Control</h2>
<h3>Nx Statistics</h3>
<h3>ExN50 Statistics</h3>
Using rcorrected merged reads for transcript abundance and Ex90N50 statistics:

```bash
mkdir kallisto_merged_reads

align_and_estimate_abundance.pl --seqType fq --transcripts filtered_cdhitted/protein_0.9/contigs_filtered_cdhit.fasta--est_method kallisto --output_dir kallisto_merged_reads --thread_count 72 --prep_reference

align_and_estimate_abundance.pl --seqType fq --transcripts filtered_cdhitted/protein_0.9/contigs_filtered_cdhit.fasta --left ../merged_rcorrected/R1_uncorr.cor.fq --right ../merged_rcorrected/R2_uncorr.cor.fq --est_method kallisto --output_dir kallisto_merged_reads --thread_count 72 --debug --trinity_mode

cd kallisto_merged_reads

abundance_estimates_to_matrix.pl --est_method kallisto --gene_trans_map none abundance.tsv

```

Did not work because of the replicate number. Using all the 8 trimmed libraries pre-aligned with <i>kallisto_script.sh</i>:

```bash

find kallisto_Sprint2/* -name 'abundance.tsv' -exec readlink -f {} >> abundances.tsv \;

abundance_estimates_to_matrix.pl --est_method kallisto --gene_trans_map none --name_sample_by_basedir abundances.tsv


count_matrix_features_given_MIN_TPM_threshold.pl kallisto.isoform.TPM.not_cross_norm | tee kallisto.isoform.TPM.not_cross_norm.counts_by_min_TPM

contig_ExN50_statistic.pl kallisto.isoform.TMM.EXPR.matrix ../filtered_cdhitted/protein_0.9/contigs_filtered_cdhit.fasta | tee ExN50.stats

plot_ExN50_statistic.Rscript ExN50.stats
```

<h3> Read Representation</h3>
Estimation of read representation in the initial and final assemblies using <b>bowtie2</b>:

```bash
mkdir QC; cd QC

bowtie2-build filtered_cdhitted/protein_0.9/contigs_filtered_cdhit.fasta assembly_bowtie.idx

bowtie2-build trinity_merged_rcorrected_assembly/Trinity.fasta raw_bowtie.idx

bowtie2 align -p 10 -q --no-unal -k 20 -x assembly_bowtie.idx -1 ../../merged_rcorrected/R1_uncorr.cor.fq -2 ../../merged_rcorrected/R2_uncorr.cor.fq  \
     2>align_stats_assembly.txt| samtools view -@10 -Sb -o bowtie2_assembly.bam

bowtie2 align -p 10 -q --no-unal -k 20 -x raw_bowtie.idx -1 ../../merged_rcorrected/R1_uncorr.cor.fq -2 ../../merged_rcorrected/R2_uncorr.cor.fq  \
     2>align_stats_raw.txt| samtools view -@10 -Sb -o bowtie2_raw.bam

```

<h3> Alignment to the reference genome </h3>

```bash

mkdir ~/Ps_transcriptome/cameor_genome_assembly

wget -P cameor_genome_assembly https://urgi.versailles.inra.fr/download/pea/Pisum_sativum_v1a.fa


minimap2 -ax splice Pisum_sativum.v1a.fa ../novel_merged_assembly/filtered_cdhitted/protein_0.9/contigs_filtered_cdhit.fasta > aln_splice.sam ###24 unmappead reads

minimap2 -ax asm5 Pisum_sativum.v1a.fa ../novel_merged_assembly/filtered_cdhitted/protein_0.9/contigs_filtered_cdhit.fasta > aln_asm.sam ### 1890 unmapped reads
```

<h3>Full-length transcript analysis using BLAST+</h3>
The commands for BLAST-based annotation are shown below. Here we demonstrate only the quality control mediate by BLASTX output:

```bash
mkdir blast_QC; cd blast_QC
analyze_blastPlus_topHit_coverage.pl ../blast_outputs/bestHits_blastx.outfm6 ../filtered_cdhitted/protein_0.9/contigs_filtered_cdhit.fasta ../blastdb/plant_seqs.fasta > blast_QC.txt
```

<h2> Transcriptome annotation </h2>
<h3>Mercator annotation</h3>
Protein sequences have been used as query. All possible options except for <b>CONSERVATIVE</b> and <b>IS_DNA</b> have been selected, BLAST significance cutoff is set as 80 (default value)
<h3>BlastKOALA annotation</h3>
Protein sequences have been used as query. Taxomomical affiliation has been set as 'Plants', search file has been set as 'family_eukaryotes'

<h3>Trinotate annotation</h3>
<h4>BLAST annotation</h4>

```bash
blastx -query ~/Ps_transcriptome/filtered_cdhitted/protein_0.9/contigs_filtered_cdhit.fasta -db ../uniprot_sprot.pep -num_threads 72 -max_target_seqs 1 -outfmt 6 -evalue 1e-5 > blastx.outfmt6
blastp -query ~/Ps_transcriptome/filtered_cdhitted/protein_0.9/proteins_filtered_cdhit.fasta -db ../uniprot_sprot.pep -num_threads 72 -max_target_seqs 1 -outfmt 6 -evalue 1e-5 > blastp.outfmt6
```
<h4>HMMer</h4>

```bash
gunzip Pfam-A.hmm.gz
hmmpress Pfam-A.hmm
cd Pea_novel_merged_assembly
hmmscan --cpu 72 --domtblout ../TrinotatePFAM.out ../Pfam-A.hmm ~/Ps_transcriptome/filtered_cdhitted/protein_0.9/proteins_filtered_cdhit.fasta > pfam.log
```
<h4>SignalP</h4>

```bash
signalp -format short -fasta ~/Ps_transcriptome/filtered_cdhitted/protein_0.9/proteins_filtered_cdhit.fasta -prefix signalp_out #launched from signalp home directory
```
<h4>TmHMM</h4>

```bash
tmhmm --short < ~/Ps_transcriptome/filtered_cdhitted/protein_0.9/proteins_filtered_cdhit.fasta > tmhmm.out
```
<h4>Data gathering</h4>

```bash
get_Trinity_gene_to_trans_map.pl ~/Ps_transcriptome/filtered_cdhitted/protein_0.9/contigs_filtered_cdhit.fasta > contigs_filtered_cdhit.fasta.gene_to_trans_map
Trinotate ../Trinotate.sqlite init --gene_to_trans_map contigs_filtered_cdhit.fasta.gene_to_trans_map --transcript_fasta ~/Ps_transcriptome/filtered_cdhitted/protein_0.9/contigs_filtered_cdhit.fasta --transdecoder_pep ~/Ps_transcriptome/filtered_cdhitted/protein_0.9/proteins_filtered_cdhit.fasta
Trinotate ../Trinotae.sqlite LOAD_swissprot_blastp blastp.outfmt6
Trinotate ../Trinotae.sqlite LOAD_swissprot_blastx blastx.outfmt6
Trinotate ../Trinotate.sqlite LOAD_custom_blast --outfmt ~/Ps_transcriptome/novel_merged_assembly/blast_outputs/bestHits_blastp.outfmt6 --prog blastp --dbtype plant_seqs
Trinotate ../Trinotate.sqlite LOAD_custom_blast --outfmt ~/Ps_transcriptome/novel_merged_assembly/blast_outputs/bestHits_blastx.outfmt6 --prog blastx --dbtype plant_seqs
Trinotate ../Trinotate.sqlite LOAD_pfam TrinotatePFAM.out
Trinotate ../Trinotate.sqlite LOAD_tmhmm tmhmm.out
Trinotate ../Trinotate.sqlite LOAD_signalp signalp_out_summary.signalp5
Trinotate ../Trinotate.sqlite report > trinotate_annotation_report.xls 
```

<h4>Checking contig and protein length</h4>

```bash
./mean_lengths.py -i contigs_filtered_cdhit.fasta
./mean_lengths.py -i proteins_filtered_cdhit.fasta
```

<h4>Checking lengths of the unblasted entries</h4>

```bash

awk '($7 == "" & $8 == ""){print $2}' > unmatched_contigs.txt
awk '($7 == "" & $8 == ""){print $4}' > unmatched_proteins.txt
./mean_lengths_local.py -i contigs_filtered_cdhit.fasta -q unmatched_contigs.txt
./mean_lengths_local.py -i proteins_filtered_cdhit.fasta -q unmatched_proteins.txt

```

<h3>eggNOG</h3>

```bash
emapper.py -i ~/Ps_transcriptome/novel_merged_assembly/filtered_cdhitted/protein_0.9/proteins_filtered_cdhit.fasta
```


<h3>Matching BLAST outputs to the human-readable format</h3>
Input files had been written out from the R session to the <b>BLAST_decoding</b> directory. The script *download_BLAST_names.py* provides human-readable BLAST annotations via fetching names by NCBI accessions produced by Trinotate.

```bash
gawk -i inplace 'BEGIN{FS=OFS="\t"} {if ($1="") $1=None}' BLASTX_identifiers
./download_BLAST_names.py -i BLASTX_identifiers -o BLASTX_descriptions.txt
```
The same operation was performed with BLASTP outputs.

<h3>Aligning external assembly to the Sprint-2 reference </h3>

```bash
cd ext_assembly ;
TransDecoder.LongOrfs -t GSE72573_Trinity_uniq.fasta ; TransDecoder.Predict -t  GSE72573_Trinity_uniq.fasta 
./filter_script.py -pr  GSE72573_Trinity_uniq.fasta/transdecoder.pep -nu  GSE72573_Trinity_uniq.fasta -o .
```

<h2>Variant Calling</h2>

```bash
mkdir VC ; cd VC
Trinity_gene_splice_modeler.py --trinity_fasta ~/Ps_transcriptome/novel_merged_assembly/filtered_cdhitted/protein_0.9/contigs_filtered_cdhit.fasta

mkdir Zhewan-1

### the following commands refer to variant calling for Zhewan-1 reads; the same pipeline has been run for Zhongwan-6 SNPs
### note that the trimmed reads for 10 and 20 (25) days had been merged in a direction-wise manner prior to increse coverage depth

~/Trinity/trinityrnaseq-Trinity-v2.8.4/Analysis/SuperTranscripts/AllelicVariants/run_variant_calling.py \
           --st_fa trinity_genes_fasta.fasta \
           --st_gtf trinity_genes.gtf \
           -p Zhewan1_R1_trimmed.fastq Zhewan1_R1_trimmed.fastq \
           -o Zhewan-1

cd Zhewan1 ; mkdir manual_filtering ;  mv output.vcf manual_filtering ; cd manual_filtering
gatk SelectVariants \
-V output.vcf \
-select-type SNP \
-O snps.vcf

gatk SelectVariants \
-V output.vcf \
-select-type INDEL \
-O indels.vcf

gatk VariantFiltration -V snps.vcf \
-filter "QD < 2.0" --filter-name "QD2" \
-filter "DP < 15" --filter-name "DP15" \
-filter "QUAL < 30.0" --filter-name "QUAL30" \
-filter "SOR > 3.0" --filter-name "SOR3" \
-filter "FS > 30.0" --filter-name "FS30" \
-filter "MQ < 40.0" --filter-name "MQ40" \
-filter "MQRankSum < -12.5" --filter-name "MQRankSum-12.5" \
-filter "ReadPosRankSum < -8.0" --filter-name "ReadPosRankSum-8" \
-O snps_filtered.vcf ; awk -F '\t' '{if($0 ~ /\#/) print; else if($7 == "PASS") print}' snps_filtered.vcf > snps_filtered_PASS_only.vcf

grep -E '^##' snps_filtered_PASS_only.vcf > snps_filtered_PASS_only_head.txt
grep -E '^TRINITY' snps_filtered_PASS_only.vcf > snps_filtered_PASS_only_headless.vcf

gatk VariantFiltration -V indels.vcf \
-filter "QD < 2.0" --filter-name "QD2" \
-filter "DP < 15" --filter-name "DP15" \
-filter "SOR > 3.0" --filter-name "SOR3" \
-filter "QUAL < 30.0" --filter-name "QUAL30" \
-filter "FS > 60.0" --filter-name "FS200" \
-filter "ReadPosRankSum < -20.0" --filter-name "ReadPosRankSum-20" \
-O indels_filtered.vcf ; awk -F '\t' '{if($0 ~ /\#/) print; else if($7 == "PASS") print}' indels_filtered.vcf > indels_filtered_PASS_only.vcf #25819 in Zhewan-1, 13944 SNPs in Zhongwan-6

./filter_SupTrans_gtf.py -e trinity_genes.vcf -vc Zhewan1/manual_filtering/snps_filtered_PASS_only_headless.vcf -o snps_reallocated_gene_to_trans_headless.vcf

cat snps_filtered_PASS_only_head.txt snps_filtered_PASS_only_headless.vcf snps_reallocated_gene_to_trans_headless.vcf > snps_reallocated_gene_to_trans.vcf

gatk VariantFiltration \
-V Zhewan1/manual_filtering/snps_reallocated_gene_to_trans.vcf \
-O Zhewan1/manual_filtering/snps_reallocated_gene_to_trans_isHet.vcf \
--genotype-filter-expression "isHet == 1" \
--genotype-filter-name "isHetFilter"

### hetero- and homozygous variants were then selected according to this filter

```

<h2> Variant Annotation </h2>

```bash
python filter_gff_for_snpEff -i Trinity.fasta.transdecoder.gff3 -o Trinity.fasta.transdecoder.gff3_updated
cp Trinity.fasta.transdecoder.gff3_updated contigs_filtered_cdhit.fasta snpEff_latest/snpEff/data/Pisum
mv snpEff_latest/snpEff/data/Pisum/Trinity.fasta.transdecoder.gff3_updated snpEff_latest/snpEff/data/Pisum/genes.gff
mv snpEff_latest/snpEff/data/Pisum/contigs_filtered_cdhit.fasta snpEff_latest/snpEff/data/Pisum/sequences.fa
#manually editing snpEff.config to add Pisum database to the list of available databases
java -jar snpEff_latest/snpEff/snpEff.jar build -gff3 Pisum
java -Xmx4g -jar snpEff_latest/snpEff/snpEff.jar -v Pisum Zhewan1/manual_filtration/snps_reallocated_gene_to_trans_Homo.vcf > Zhewan1/manual_filtration/snps_annotated_Homo.vcf
java -Xmx4g -jar snpEff_latest/snpEff/snpEff.jar -v Pisum Zhewan1/manual_filtration/snps_reallocated_gene_to_trans_Het.vcf > Zhewan1/manual_filtration/snps_annotated_Het.vcf

```

<h4>28.11.19, 07:11</h4> performing the same pipeline onto the Sprint-2 SNPs